<a href="https://colab.research.google.com/github/tomknightatl/usccb-parish-extraction/blob/main/notebooks/00_Colab_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# USCCB Parish Extraction - Google Colab Setup

This notebook sets up the environment for parish extraction in Google Colab.

## 🔑 Required Secrets
**Before running, add these to your Colab Secrets (🔑 icon in sidebar):**
- `SUPABASE_URL` - Your Supabase project URL
- `SUPABASE_KEY` - Your Supabase API key
- `GENAI_API_KEY_USCCB` - Your Google AI API key

## 📋 Setup Steps
1. **Cell 1**: Install dependencies (dependency conflicts are normal)
2. **Cell 2**: Setup paths and test imports
3. **Cell 3**: Configure API keys
4. **Cell 4**: Verify everything works

💡 **Tip**: If you see import errors, restart runtime and re-run from Cell 2.

In [ ]:
# Cell 1: Clone Repository and Install Dependencies
import os
import sys

print("📁 Setting up repository...")

# Clone or update repository
if not os.path.exists('/content/usccb-parish-extraction'):
    !git clone https://github.com/tomknightatl/usccb-parish-extraction.git
    print("✅ Repository cloned")
else:
    print("✅ Repository already exists")
    os.chdir('/content/usccb-parish-extraction')
    !git pull --quiet
    print("✅ Repository updated")

# Change to repo directory
os.chdir('/content/usccb-parish-extraction')
print(f"📂 Working in: {os.getcwd()}")

print("\n📦 Installing additional packages...")
print("⚠️  Dependency conflicts with Colab are normal and can be ignored")

# Install only the packages we need (avoid conflicts with Colab built-ins)
!pip install --quiet selenium==4.15.0 webdriver-manager==4.0.1
!pip install --quiet beautifulsoup4==4.12.2 lxml
!pip install --quiet google-generativeai==0.3.0 tenacity==8.2.3
!pip install --quiet "supabase>=2.15.0"

print("✅ Package installation completed")
print("\n💡 If you see warnings about package conflicts, they can be safely ignored.")
print("   Colab has built-in versions of pandas, requests, and numpy that work fine.")

# Check if restart is needed
try:
    import google.generativeai as genai
    print("\n🎉 All packages ready! You can continue to the next cell.")
except ImportError:
    print("\n🔄 Import error detected. Please:")
    print("   1. Go to Runtime → Restart runtime")
    print("   2. Skip this cell and run from Cell 2")

📁 Setting up repository...
Cloning into 'usccb-parish-extraction'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (160/160), done.
remote: Compressing objects: 100% (148/148), done.
remote: Total 160 (delta 73), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (160/160), 67.93 KiB | 2.72 MiB/s, done.
Resolving deltas: 100% (73/73), done.
✅ Repository cloned
📂 Working in: /content/usccb-parish-extraction

📦 Installing additional packages...
⚠️  Dependency conflicts with Colab are normal and can be ignored
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.9/145.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [ ]:
# Cell 2: Setup Environment and Test Imports
import sys
import os

# Ensure we're in the right directory
repo_path = '/content/usccb-parish-extraction'
if not os.path.exists(repo_path):
    print("❌ Repository not found. Please run Cell 1 first.")
    raise FileNotFoundError("Repository not cloned")

os.chdir(repo_path)
if repo_path not in sys.path:
    sys.path.append(repo_path)

print(f"📂 Working directory: {os.getcwd()}")
print("🐍 Python path configured")

# Test core package imports
print("\n🧪 Testing package imports...")

try:
    # Test built-in Colab packages
    import pandas as pd
    import requests
    import numpy as np
    print(f"✅ Pandas {pd.__version__} (Colab built-in)")
    print(f"✅ Requests {requests.__version__} (Colab built-in)")

    # Test newly installed packages
    import selenium
    from selenium import webdriver
    print(f"✅ Selenium {selenium.__version__}")

    import bs4
    print(f"✅ BeautifulSoup {bs4.__version__}")

    import google.generativeai as genai
    print("✅ Google Generative AI")

    import supabase
    print(f"✅ Supabase {supabase.__version__}")

    print("\n🎉 All core packages imported successfully!")

except ImportError as e:
    print(f"❌ Import error: {e}")
    print("\n🔧 Troubleshooting:")
    print("   1. Restart runtime: Runtime → Restart runtime")
    print("   2. Re-run from this cell (skip Cell 1)")
    print("   3. If still failing, re-run Cell 1 then this cell")
    raise

📂 Working directory: /content/usccb-parish-extraction
🐍 Python path configured

🧪 Testing package imports...
✅ Pandas 2.2.2 (Colab built-in)
✅ Requests 2.32.3 (Colab built-in)
✅ Selenium 4.14.0
✅ BeautifulSoup 4.12.2
✅ Google Generative AI
✅ Supabase 2.15.2

🎉 All core packages imported successfully!


In [ ]:
# Cell 3: Configure API Keys and Test Connections
from google.colab import userdata
from config.settings import setup_environment, set_config

def get_secret_safely(key):
    """Safely retrieve a secret from Colab"""
    try:
        value = userdata.get(key)
        return value if value else None
    except Exception as e:
        print(f"⚠️  Could not retrieve '{key}': {e}")
        return None

print("🔑 Retrieving API keys from Colab Secrets...")

# Get API keys
supabase_url = get_secret_safely('SUPABASE_URL')
supabase_key = get_secret_safely('SUPABASE_KEY')
genai_key = get_secret_safely('GENAI_API_KEY_USCCB')

# Check what we have
print("\n📋 API Key Status:")
print(f"   {'✅' if supabase_url else '❌'} SUPABASE_URL")
print(f"   {'✅' if supabase_key else '❌'} SUPABASE_KEY")
print(f"   {'✅' if genai_key else '❌'} GENAI_API_KEY_USCCB")

# Setup configuration
if supabase_url or supabase_key or genai_key:
    print("\n⚙️  Setting up configuration...")
    try:
        config = setup_environment(
            supabase_url=supabase_url,
            supabase_key=supabase_key,
            genai_api_key=genai_key,
            max_dioceses=3  # Start with small number for testing
        )

        set_config(config)
        print("\n✅ Configuration complete!")

        # Show what's configured
        print(f"\n🔧 Configuration Summary:")
        print(f"   📊 Database: {'Connected' if config.supabase else 'Not available'}")
        print(f"   🤖 AI Analysis: {'Enabled' if config.genai_enabled else 'Mock mode'}")
        print(f"   🎯 Max dioceses: {config.max_dioceses}")

    except Exception as e:
        print(f"\n❌ Configuration error: {e}")
        print("\n🔧 Troubleshooting:")
        print("   • Check that Supabase URL starts with 'https://'")
        print("   • Verify Google AI key starts with 'AIza'")
        print("   • Ensure all keys are valid and have proper permissions")
        config = None

else:
    print("\n❌ No API keys found!")
    print("\n🔧 Setup Instructions:")
    print("   1. Click the 🔑 icon in the left sidebar")
    print("   2. Add your three API keys:")
    print("      • SUPABASE_URL")
    print("      • SUPABASE_KEY")
    print("      • GENAI_API_KEY_USCCB")
    print("   3. Enable 'Notebook access' for each key")
    print("   4. Re-run this cell")
    config = None

🔑 Retrieving API keys from Colab Secrets...

📋 API Key Status:
   ✅ SUPABASE_URL
   ✅ SUPABASE_KEY
   ✅ GENAI_API_KEY_USCCB

⚙️  Setting up configuration...
✅ Supabase connected and tested
✅ Google AI configured and tested

✅ Configuration complete!

🔧 Configuration Summary:
   📊 Database: Connected
   🤖 AI Analysis: Enabled
   🎯 Max dioceses: 3


In [ ]:
# Cell 4: Verify Complete Installation
print("🧪 Running comprehensive system verification...\n")

try:
    # Test project imports
    from src.models import Parish, ExtractionResult, SiteType, Diocese
    from src.utils.webdriver import clean_text, extract_phone
    from src.utils.ai_analysis import analyze_with_ai, detect_site_type
    from src.extractors import get_extractor
    from src.pipeline import ParishExtractionPipeline

    print("✅ All project modules imported successfully")

    # Test core functionality
    print("\n🔧 Testing core functions...")

    # Test data models
    test_parish = Parish(name="St. Test Parish", city="Test City")
    print(f"✅ Parish model: {test_parish}")

    # Test utilities
    cleaned = clean_text("  Test   Text  ")
    assert cleaned == "Test Text"
    print("✅ Text cleaning utility")

    phone = extract_phone("Call (555) 123-4567 today")
    assert phone == "(555) 123-4567"
    print("✅ Phone extraction utility")

    # Test extractors
    extractor = get_extractor('generic')
    print(f"✅ Extractors: {extractor.name}")

    # Test AI analysis
    ai_result = analyze_with_ai("parish directory test", "parish_directory")
    print(f"✅ AI analysis (score: {ai_result.get('score', 'N/A')})")

    if config:
        # Test pipeline
        pipeline = ParishExtractionPipeline(config)
        print("✅ Pipeline initialization")

        # Test database connection
        if config.supabase:
            try:
                # Simple connection test
                config.supabase.table('Dioceses').select('Name').limit(1).execute()
                print("✅ Database connection verified")
            except Exception as e:
                print(f"⚠️  Database test: {e}")
                print("   (This may be normal if tables don't exist yet)")

        print("\n🎉 SETUP COMPLETE! Everything is working perfectly.")
        print("\n🚀 Ready for parish extraction!")
        print("\n📚 Next Steps:")
        print("   1. 🎯 Quick Demo: Open and run '99_Simple_Demo.ipynb'")
        print("   2. 📋 Full Setup: Run '01_Build_Dioceses_Database.ipynb'")
        print("   3. 🔍 Advanced: Explore individual extraction notebooks")

    else:
        print("\n⚠️  Setup incomplete - missing API keys")
        print("\n✅ Good news: Core functionality works!")
        print("📝 You can still test extraction with mock data")
        print("\n🔧 To enable full features:")
        print("   • Add your API keys to Colab Secrets")
        print("   • Re-run Cell 3")

except Exception as e:
    print(f"❌ Verification failed: {e}")
    print("\n🔧 Troubleshooting steps:")
    print("   1. Restart runtime: Runtime → Restart runtime")
    print("   2. Re-run this entire notebook")
    print("   3. Check that all files downloaded correctly")
    raise

🧪 Running comprehensive system verification...

✅ All project modules imported successfully

🔧 Testing core functions...
✅ Parish model: Parish(St. Test Parish, Test City)
✅ Text cleaning utility
✅ Phone extraction utility
✅ Extractors: GenericExtractor
✅ AI analysis (score: 10)
✅ Pipeline initialization
✅ Database connection verified

🎉 SETUP COMPLETE! Everything is working perfectly.

🚀 Ready for parish extraction!

📚 Next Steps:
   1. 🎯 Quick Demo: Open and run '99_Simple_Demo.ipynb'
   2. 📋 Full Setup: Run '01_Build_Dioceses_Database.ipynb'
   3. 🔍 Advanced: Explore individual extraction notebooks


## 🎉 Setup Complete!

If you see "SETUP COMPLETE" above, you're ready to extract parish data!

### 🚀 Quick Start Options:

1. **🎯 Instant Demo** (5 minutes)
   - Open [`99_Simple_Demo.ipynb`](99_Simple_Demo.ipynb)
   - Run all cells to see the system in action

2. **📋 Complete Setup** (20 minutes)
   - Run [`01_Build_Dioceses_Database.ipynb`](01_Build_Dioceses_Database.ipynb) first
   - Then run other notebooks as needed

### 🛠️ Troubleshooting:

**"Import Error" or "Module Not Found":**
- Restart runtime: `Runtime` → `Restart runtime`
- Re-run this notebook from Cell 2

**"No API keys found":**
- Click 🔑 in sidebar → Add your keys → Re-run Cell 3

**"Database connection failed":**
- Check your Supabase URL and key
- Ensure your Supabase project is active

---

💡 **Tip**: Bookmark this notebook for easy re-setup in future sessions!